<a href="https://colab.research.google.com/github/hufahamdika/BigData_LSH_using_ApacheSpark/blob/main/LSH_Apache_Spark_5025201197_Abidjanna_Zulfa_Hamdika.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignments 3
- Abidjanna Zulfa Hamdika
- 5025201197
- Big Data A

## Source:
1. https://www.uber.com/en-ID/blog/lsh/
2. https://stackoverflow.com/questions/56816537/cant-find-kaggle-json-file-in-google-colab
3. https://spark.apache.org/docs/latest/api/python/index.html
4. https://spark.apache.org/docs/latest/ml-features.html#locality-sensitive-hashing

# Initialization

## Checking the Environment

In [1]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.9.16


## Installing Apache Spark (PySpark)

In [2]:
## Installing Apache Spark (PySpark)

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=b96b0b2fcb4f098fbfda06b704b66fa8c62d094b60f12542e2398bd7728be23a
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


## Initialize Apache Spark context

In [3]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
# Check spark session
print(spark)

# Data Mining Task

The LSH task always consists of three steps:

1. Converting original data into vectors
2. Calculate the hash using MinHash algorithm
3. Searching the similar pairs using k-Nearest Neighbor, or join algorithm.

## Downloading the dataset

In [5]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!mkdir ~/.kaggle/
!touch ~/.kaggle/kaggle.json

In [8]:
# PLEASE USE YOUR OWN KEY
# Download your own key according to this instruction https://github.com/Kaggle/kaggle-api#api-credentials

import json
api_token = {"username":"marvesco","key":"5969b2bff995464adc703b3d908f0169"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json


In [10]:
# Download from https://www.kaggle.com/datasets/urbanbricks/wikipedia-promotional-articles

!kaggle datasets download -d urbanbricks/wikipedia-promotional-articles

 99% 200M/201M [00:06<00:00, 34.9MB/s]
100% 201M/201M [00:06<00:00, 34.2MB/s]


## Extract Dataset

In [11]:
!unzip wikipedia-promotional-articles.zip

Archive:  wikipedia-promotional-articles.zip
  inflating: good.csv                
  inflating: promotional.csv         


## check files in current directory

In [12]:
!ls -la

total 783140
drwxr-xr-x 1 root root      4096 Mar  9 17:27 .
drwxr-xr-x 1 root root      4096 Mar  9 17:24 ..
drwxr-xr-x 4 root root      4096 Mar  7 18:12 .config
-rw-r--r-- 1 root root 475685227 Oct 27  2019 good.csv
-rw-r--r-- 1 root root 115360355 Oct 27  2019 promotional.csv
drwxr-xr-x 1 root root      4096 Mar  7 18:14 sample_data
-rw-r--r-- 1 root root 210863294 Mar  9 17:27 wikipedia-promotional-articles.zip


## Read the dataset

In [13]:
# Read CSV (promotional.csv)
df = spark.read.option("header", True).csv("/content/promotional.csv")
df.printSchema()

root
 |-- text: string (nullable = true)
 |-- advert: string (nullable = true)
 |-- coi: string (nullable = true)
 |-- fanpov: string (nullable = true)
 |-- pr: string (nullable = true)
 |-- resume: string (nullable = true)
 |-- url: string (nullable = true)



In [14]:
# Add an ID for the dataset
from pyspark.sql.functions import monotonically_increasing_id

newsDF = df.withColumn("id", monotonically_increasing_id())
newsDF.show()

+--------------------+------+---+------+---+------+--------------------+---+
|                text|advert|coi|fanpov| pr|resume|                 url| id|
+--------------------+------+---+------+---+------+--------------------+---+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|
|2N Telekomunikace...|     1|  0|     0|  0|     0|https://en.wikipe...|  7|
|A 3D printing mar...|     1|  0|     0|  0|     0|https://en.wikipe...|  8|
|3DR is an America...|     1|  1|     0|  0|     0|https://en.wikipe...|  9|

In [15]:
# Get the totals row
newsDF.count()

23837

## 1. Prepare the tokenizer
We transform the input into tokenized words

In [16]:
# Prepare the tokenizer
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# tokenize text into dataframe
wordsDF = tokenizer.transform(newsDF)
wordsDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|[1st, round, ente...|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|[2ergo, is, a, pr...|
|2N Telekomunikace...| 

In [17]:
# Vectorize the dataset
from pyspark.ml.feature import CountVectorizer

# create a model and fit it into the tokenized data
vocabSize=1000
cvModel = CountVectorizer(inputCol="words", outputCol="features", vocabSize=vocabSize, minDF=10).fit(wordsDF)

# transform data into vector
vectorizedDF = cvModel.transform(wordsDF)
vectorizedDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(1000,[0,1,2,3,4,...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(1000,[0,1,2,3,4,...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(1000,[0,1,2,3,4,...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(1000,[0,1,2,3,4,...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|(1000,[0,1,2,3,4,...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|

## 2. Fit/train an LSH Model

In [18]:
from pyspark.ml.feature import MinHashLSH

# Define the MinHashLSH model
mh = MinHashLSH(inputCol="features", outputCol="hashValues", numHashTables=3)

# Train the model and apply model
LSHmodel = mh.fit(vectorizedDF)
LSHmodel.transform(vectorizedDF).show()


+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|          hashValues|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(1000,[0,1,2,3,4,...|[[7459245.0], [96...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(1000,[0,1,2,3,4,...|[[5.4944481E7], [...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(1000,[0,1,2,3,4,...|[[1.0033226E7], [...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(1000,[0,1,2,3,4,...|[[4500738.0], [96...|
|The Jerusalem Bie...| 

## 3. Searching the similar pairs/items for a key "united" "states"

In [19]:
# Testing searching for "united" "states"
print(cvModel.vocabulary.index("united"))
print(cvModel.vocabulary.index("states"))

92
198


In [20]:
# Convert the input with 2 words into a 1000-size vector
# If the words exist in the index, we will give the value 1.0, otherwise 0.0
# Final result: key = [0, 0, ..., 1.0, ..., 1.0, 0, ..., 0]
from pyspark.ml.linalg import Vectors
key = Vectors.sparse(vocabSize, {cvModel.vocabulary.index("united"): 1.0, cvModel.vocabulary.index("states"): 1.0})

In [25]:
# Define the number of neighbors
k = 40
# Search inside the LSH model that we already trained
resultDF = LSHmodel.approxNearestNeighbors(vectorizedDF, key, k)
resultDF.show()

+--------------------+------+---+------+---+------+--------------------+----------+--------------------+--------------------+--------------------+------------------+
|                text|advert|coi|fanpov| pr|resume|                 url|        id|               words|            features|          hashValues|           distCol|
+--------------------+------+---+------+---+------+--------------------+----------+--------------------+--------------------+--------------------+------------------+
|Phyllis C. Borzi ...|     0|  0|     0|  0|     1|https://en.wikipe...|      2593|[phyllis, c., bor...|(1000,[0,2,3,7,8,...|[[4.64411439E8], ...|               0.8|
|Metech Recycling ...|     1|  0|     0|  0|     0|https://en.wikipe...|8589935638|[metech, recyclin...|(1000,[0,1,2,3,5,...|[[2.65210448E8], ...|0.8823529411764706|
|Calvary Christian...|     1|  0|     0|  0|     0|https://en.wikipe...|      3149|[calvary, christi...|(1000,[0,3,5,6,35...|[[1.80151374E8], ...|               0.9|
|Mar

In [23]:
# Save the result into CSV
import pandas as pd
data = resultDF.toPandas()
data.to_csv("result.csv")


## Check the Result

In [27]:
import pandas as pd

df = pd.read_csv('result.csv')
df.head(10)

,Unnamed: 0,text,advert,coi,fanpov,pr,resume,url,id,words,features,hashValues,distCol
0,0,Phyllis C. Borzi was the Obama administration'...,0,0,0,0,1,https://en.wikipedia.org/wiki/Phyllis%20Borzi,2593,"['phyllis', 'c.', 'borzi', 'was', 'the', 'obam...","(1000,[0,2,3,7,8,92,194,198,515,593],[4.0,3.0,...","[DenseVector([464411439.0]), DenseVector([2337...",0.800000
1,1,Metech Recycling is a United States based elec...,1,0,0,0,0,https://en.wikipedia.org/wiki/Metech%20Incorpo...,8589935638,"['metech', 'recycling', 'is', 'a', 'united', '...","(1000,[0,1,2,3,5,6,11,15,16,41,55,59,92,116,19...","[DenseVector([265210448.0]), DenseVector([1354...",0.882353
2,2,"Calvary Christian High School is a private, co...",1,0,0,0,0,https://en.wikipedia.org/wiki/Calvary%20Christ...,3149,"['calvary', 'christian', 'high', 'school', 'is...","(1000,[0,3,5,6,35,73,89,92,707],[1.0,2.0,1.0,1...","[DenseVector([180151374.0]), DenseVector([1479...",0.900000
3,3,Redrock Broadcasting is a media company headqu...,1,0,0,0,0,https://en.wikipedia.org/wiki/Redrock%20Broadc...,8589939640,"['redrock', 'broadcasting', 'is', 'a', 'media'...","(1000,[0,3,5,6,41,92,167,309,322,707],[2.0,2.0...","[DenseVector([424648146.0]), DenseVector([1479...",0.909091
4,4,Marine Corps Installations West MCI WEST is th...,1,0,0,0,0,https://en.wikipedia.org/wiki/Marine%20Corps%2...,8589935096,"['marine', 'corps', 'installations', 'west', '...","(1000,[0,1,2,3,4,6,10,11,58,92,129,198,211,238...","[DenseVector([68583438.0]), DenseVector([23377...",0.909091
5,5,"Blythe is an English surname. As of 2014, 58.1...",0,0,1,0,0,https://en.wikipedia.org/wiki/Blythe%20%28surn...,2454,"['blythe', 'is', 'an', 'english', 'surname.', ...","(1000,[0,1,2,3,6,8,9,16,25,27,29,34,38,66,92,1...","[DenseVector([105772750.0]), DenseVector([1102...",0.913043
6,6,Jason Fairbourne is a United States business c...,0,0,0,1,0,https://en.wikipedia.org/wiki/Jason%20Fairbourne,6688,"['jason', 'fairbourne', 'is', 'a', 'united', '...","(1000,[0,1,2,3,5,6,7,9,12,14,16,17,35,67,92,10...","[DenseVector([81837869.0]), DenseVector([10340...",0.920000
7,7,The EcoBroker designation is an environmental ...,1,0,0,0,0,https://en.wikipedia.org/wiki/EcoBroker,5974,"['the', 'ecobroker', 'designation', 'is', 'an'...","(1000,[0,1,2,3,5,6,7,12,16,50,92,93,107,111,13...","[DenseVector([36542088.0]), DenseVector([47347...",0.925926
8,8,This is a list of television programmes curren...,0,0,1,0,0,https://en.wikipedia.org/wiki/List%20of%20BBC%...,1922,"['this', 'is', 'a', 'list', 'of', 'television'...","(1000,[0,1,2,3,5,6,11,28,92,262,359,678,863],[...","[DenseVector([110920712.0]), DenseVector([1479...",0.928571
9,9,"TIMGlobal Media, formerly Thomas Industrial Me...",1,0,0,0,0,https://en.wikipedia.org/wiki/TIMGlobal%20Media,8589943474,"['timglobal', 'media,', 'formerly', 'thomas', ...","(1000,[0,1,2,3,5,6,8,19,24,25,27,30,41,82,92,9...","[DenseVector([57902988.0]), DenseVector([59927...",0.931034
